In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/Github/SSM/")
# Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia/")

  Activating project at `\Users\ryansenne\Documents\Github\SSM`


In [2]:
include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
# include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using Random
using .SSM
using Distributions

### Gaussian GLM Example

In [3]:
# Fit a Gaussian GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]
y = X_concat * β + rand(Normal(0, 1), 10000)

# define a GLM
glm = GaussianRegression(X, y)
fit!(glm, 1000)

4-element Vector{Float64}:
  1.963508062948609
  3.0472521214354646
 -4.032847152584493
  5.085586252239004

### Poisson GLM Example

In [4]:
# Fit a Poisson GLM
# Generate some synthetic data
X = rand(10000, 3);  # 100 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Poisson.(exp.(X_concat * β))))

# define a GLM
glm = PoissonRegression(X, y)
fit!(glm, 1000)


4-element Vector{Float64}:
  1.9972876910665238
  3.0018761978448465
 -4.001568346180793
  5.002617122785103

### Binomial GLM Example (i.e. logistic regression)

In [5]:
# sigmoid function
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

# Fit a Binomial GLM
# Generate some synthetic data
X = rand(10000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(10000), X)
β = [2, 3, -4, 5]

# Generate some synthetic data
y = Vector{Float64}(rand.(Binomial.(1, sigmoid.(X_concat * β))))

# define a GLM
glm = BinomialRegression(X, y)
fit!(glm, 1000)

MethodError: MethodError: no method matching BinomialRegression(::Matrix{Float64}, ::Vector{Float64}, ::Vector{Float64}, ::LogitLink, ::CrossEntropyLoss)

Closest candidates are:
  BinomialRegression(::Matrix{T}, ::AbstractVector, !Matched::Bool, ::Link, ::Loss) where T<:Real
   @ Main.SSM \Users\ryansenne\Documents\Github\SSM\src\Regression.jl:231
  BinomialRegression(::Matrix{T}, ::AbstractVector, !Matched::Bool, ::Link) where T<:Real
   @ Main.SSM \Users\ryansenne\Documents\Github\SSM\src\Regression.jl:231
  BinomialRegression(::Matrix{T}, ::AbstractVector) where T<:Real
   @ Main.SSM \Users\ryansenne\Documents\Github\SSM\src\Regression.jl:231
  ...


# Autoregressive Model

In [3]:
# simulate AR(1) model
init_val = 0.0
phi = 0.9
sigma = 1.0
T = 1000
y = Vector{Float64}(undef, T)
y[1] = init_val
for t in 2:T
    y[t] = phi * y[t-1] + rand(Normal(0, sigma))
end

ar_1 = Autoregression(y, 1)
fit!(ar_1)

0.9538958584342558

In [15]:
# try out markov glm

# Generate synthetic data from two distinct regimes
X = rand(1000, 3); # 1000 samples, 3 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(1000), X)
β₁ = [-20, -30, -40, 50]
β₂ = [6, 7, 8, -9]

# Generate some synthetic data
y₁ = X_concat * β₁ + rand(Normal(0, 0.1), 1000)
y₂ = X_concat * β₂ + rand(Normal(0, 1), 1000)

# concatenate X1 and X2
x = vcat(X, X)

# concatenate y1 and y2
y = vcat(y₁, y₂)

# define a Gaussian HMMGLM
glm = SwitchingGaussianRegression(y, x, 2)

MarkovRegressionEM(glm)

Iteration: 1, Log-likelihood: -8924.586743524973
Iteration: 2, Log-likelihood: -8739.362326611385
Iteration: 3, Log-likelihood: -8590.645568136204
Iteration: 4, Log-likelihood: -8487.922486518475
Iteration: 5, Log-likelihood: -8425.134856905965
Iteration: 6, Log-likelihood: -8388.1560823782
Iteration: 7, Log-likelihood: -8366.508137598063
Iteration: 8, Log-likelihood: -8354.032129290421
Iteration: 9, Log-likelihood: -8347.05339618205
Iteration: 10, Log-likelihood: -8343.28945695505
Iteration: 11, Log-likelihood: -8341.336144244464
Iteration: 12, Log-likelihood: -8340.358723752062
Iteration: 13, Log-likelihood: -8339.884591585684
Iteration: 14, Log-likelihood: -8339.660157362405
Iteration: 15, Log-likelihood: -8339.55583983571
Iteration: 16, Log-likelihood: -8339.507983863976
Convergence reached at iteration 16
3.0470238249608047e12
2.736186558656916e12
Initial Variance: [1.5242740495051548e9, 1.3687776681625392e9]
NaN
NaN
Log-likelihood at iteration 1: -1.661341605283923e7
NaN
NaN
Log-

In [16]:
glm.B[2].regression_model.β

4-element Vector{Float64}:
 -19.8170073700559
 -28.185869452125555
 -37.88979160361808
  47.35209106013571

In [4]:
glm.σ²

2-element Vector{Float64}:
 NaN
 NaN

In [5]:
glm.B[2].regression_model.β

4-element Vector{Float64}:
 -20.058395420154884
 -28.413295346218813
 -38.62800924981708
  48.29198573140382

In [3]:
function sigmoid(x)
    return 1 / (1 + exp(-x))
end

# generate observations from a two state hmm-glm with bernoulli emissions
X = rand(1000, 2); # 1000 samples, 1 features;
# Add a column of ones to represent the intercept term
X_concat = hcat(ones(1000), X)
β₁ = [0, 1, 2]
β₂ = [-1, -1, -1]

# Generate some synthetic data
y₁ = Vector{Float64}(rand.(Bernoulli.(sigmoid.(X_concat[1:500, :] * β₁))))
y₂ = Vector{Float64}(rand.(Bernoulli.(sigmoid.(X_concat[501:end, :] * β₂))))

# concatenate ys
y = vcat(y₁, y₂)

# define a Bernoulli HMMGLM
hmmglm = SwitchingBinomialRegression(Int.(y), X_concat[:, 2:3], 2)

MethodError: MethodError: no method matching Vector{Float64}(::Matrix{Float64})

Closest candidates are:
  Array{T, N}(::AbstractArray{S, N}) where {T, N, S}
   @ Base array.jl:621
  Vector{T}() where T
   @ Core boot.jl:496
  Array{T, N}(!Matched::Core.Compiler.BitArray{N}) where {T, N}
   @ Core bitarray.jl:495
  ...


In [4]:
y

1000-element Vector{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0